In [1]:
!apt update
!apt install xvfb

from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append("/content/drive/MyDrive/Colab Notebooks/my_modules/")
%cd /content/drive/MyDrive/spring_seminar/drone_sim_pybullet

Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:2 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:4 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main Sources [1,746 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:12 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main amd64 Packages [893 kB]
Ign:13 https://developer.download.nvidia.com/compute

In [7]:
!git status
!git add my_codes/*

On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
  (use "git add/rm <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   experiments/learning/sa_script.bash
	modified:   experiments/performance/parallelism.sh
	modified:   gym_pybullet_drones/envs/BaseAviary.py
	modified:   my_codes/env_test.ipynb
	deleted:    openaigym.episode_batch.0.20896.stats.json
	deleted:    openaigym.manifest.0.20896.manifest.json
	modified:   ros2/ros2_and_pkg_setups.zsh

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	my_codes/copy_day3.ipynb
	openaigym.episode_batch.1.4741.stats.json
	openaigym.episode_batch.2.4741.stats.json
	openaigym.manifest.1.4741.manifest.json
	openaigym.manifest.2.4741.manifest.json

no changes added to commit (use "git add" and/or "git commit -a")


In [9]:
!git config --global user.email "exmcube@gmail.com"
!git config --global user.name "takepiyo"
!git commit -m "colab revise"

[master c349d28] colab revise
 2 files changed, 2 insertions(+), 499 deletions(-)
 create mode 100644 my_codes/copy_day3.ipynb
 rewrite my_codes/env_test.ipynb (99%)


In [10]:
!git push origin master

fatal: could not read Username for 'https://github.com': No such device or address


In [1]:
import math
import numpy as np
from scipy.signal import lfilter
import gym
import gnwrapper
from cpprb import ReplayBuffer
import torch
from torch import nn

from stable_baselines3 import A2C
from stable_baselines3.a2c import MlpPolicy
from stable_baselines3.common.env_checker import check_env

from gym_pybullet_drones.utils.Logger import Logger
from gym_pybullet_drones.envs.single_agent_rl.TakeoffAviary import TakeoffAviary
from gym_pybullet_drones.utils.utils import sync, str2bool

In [2]:
!nvidia-smi

/bin/bash: nvidia-smi: コマンドが見つかりません


In [3]:
# ドローンをランダムに行動させ制御対象を把握する
monitor_env = gym.make("takeoff-aviary-v0", gui=True)
episode_max_steps = 500

for episode_idx in range():
    monitor_env.reset()
    total_rew = 0.
    for _ in range(episode_max_steps):
        _, rew, done, _ = monitor_env.step(monitor_env.action_space.sample())
        total_rew += rew
        if done:
            break
    print("iter={0: 3d} total reward: {1: 4.4f}".format(
        episode_idx, total_rew))
    
env = gym.make("takeoff-aviary-v0")

[INFO] BaseAviary.__init__() loaded parameters from the drone's .urdf:
[INFO] m 0.027000, L 0.039700,
[INFO] ixx 0.000014, iyy 0.000014, izz 0.000022,
[INFO] kf 0.000000, km 0.000000,
[INFO] t2w 2.250000, max_speed_kmh 30.000000,
[INFO] gnd_eff_coeff 11.368590, prop_radius 0.023135,
[INFO] drag_xy_coeff 0.000001, drag_z_coeff 0.000001,
[INFO] dw_coeff_1 2267.180000, dw_coeff_2 0.160000, dw_coeff_3 -0.110000
viewMatrix (-0.8660253882408142, -0.2499999701976776, 0.4330126941204071, 0.0, 0.0, 0.8660253286361694, 0.4999999701976776, 0.0, -0.4999999701976776, 0.4330126643180847, -0.75, 0.0, -0.0, 5.960464477539063e-08, -2.999999761581421, 1.0)
projectionMatrix (1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, -1.0000200271606445, -1.0, 0.0, 0.0, -0.02000020071864128, 0.0)
/home/takeshi/.pyenv/versions/gym_drone_pybullet/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), '

In [5]:
def angle_normalize(x):
    return ((x + np.pi) % (2 * np.pi)) - np.pi
 

# def reward_fn(obses, acts):
#     is_single_input = obses.ndim == acts.ndim and acts.ndim == 1
#     if is_single_input:
#         thetas = np.arctan2(obses[1], obses[0])
#         theta_dots = obses[2]
#     else:
#         assert obses.ndim == acts.ndim == 2
#         assert obses.shape[0] == acts.shape[0]
#         acts = np.squeeze(acts)
#         thetas = np.arctan2(obses[:, 1], obses[:, 0])
#         theta_dots = obses[:, 2]
#         assert thetas.shape == theta_dots.shape == acts.shape

#     acts = np.clip(acts, -2, 2)
#     costs = angle_normalize(thetas) ** 2 + .1 * theta_dots ** 2 + .001 * (acts ** 2)

#     return -costs

def reward_fn(obses, acts):
    pos, ang, vel, gyr = obses[:, 0:3], angle_normalize(obses[:, 3:6]), obses[:, 6:9], angle_normalize(obses[:, 9:12])
    cost = np.zeros((pos.shape[0], ))
    cost += np.sum(ang[:, 0:2] ** 2, axis=1)
    cost += np.sum(gyr ** 2, axis=1) * 0.1
    cost += np.sum(acts ** 2, axis=1) * 0.001
    cost += (pos[:, 2] < 0.1).astype(np.float32)
    return -cost

In [ ]:
# ダイナミクスモデル
class DynamicsModel(torch.nn.Module):
    def __init__(self, input_dim, output_dim, units=(32, 32)):
        super().__init__()

        # 隠れ層2層のMLP
        self.model = torch.nn.Sequential(
            torch.nn.Linear(input_dim, units[0]),
            torch.nn.ReLU(),
            torch.nn.Linear(units[0], units[1]),
            torch.nn.ReLU(),
            torch.nn.Linear(units[1], output_dim)
        )

        self._loss_fn = torch.nn.MSELoss(reduction='mean')
        self._optimizer = torch.optim.Adam(self.model.parameters(), lr=0.001)

    def predict(self, inputs):
        assert inputs.ndim == 2
        return self.model(inputs)

    def fit(self, inputs, labels):
        predicts = self.predict(inputs)
        loss = self._loss_fn(predicts, labels)
        self._optimizer.zero_grad()
        loss.backward()
        self._optimizer.step()
        return loss.data.numpy()

obs_dim = env.observation_space.high.size
act_dim = env.action_space.high.size

dynamics_model = DynamicsModel(input_dim=obs_dim + act_dim, output_dim=obs_dim)


# [課題1] ダイナミクスモデルを用いた次状態の予測
def predict_next_state(obses, acts):
    assert obses.shape[0] == acts.shape[0]
    # ダイナミクスモデルへの入力は状態と行動の Concata
    inputs = np.concatenate([obses, acts], axis=1)
    inputs = torch.from_numpy(inputs).float()

    # ダイナミクスモデルの出力は次の状態と現在の状態との差分
    obs_diffs = dynamics_model.predict(inputs).data.numpy()
    assert obses.shape == obs_diffs.shape
    next_obses = obses + obs_diffs
    return next_obses

In [ ]:
# 10Kデータ分 (s, a, s') を保存できるリングバッファを用意します
rb_dict = {
    "size": 10000,
    "default_dtype": np.float32,
    "env_dict": {
        "obs": {"shape": env.observation_space.shape},
        "next_obs": {"shape": env.observation_space.shape},
        "act": {"shape": env.action_space.shape}}}
dynamics_buffer = ReplayBuffer(**rb_dict)

In [ ]:
# ランダム方策
class RandomPolicy:
    def __init__(self, max_action, act_dim):
        self._max_action = max_action  # action の最大値
        self._act_dim = act_dim  # action の次元数

    def get_actions(self, batch_size):
        # 一様分布からバッチサイズ分ランダムにサンプリング
        return np.random.uniform(
            low=-self._max_action,
            high=self._max_action,
            size=(batch_size, self._act_dim))

policy = RandomPolicy(
    max_action=env.action_space.high[0],
    act_dim=env.action_space.high.size)


# [課題2] RSの実装
def random_shooting(init_obs, n_mpc_episodes=64, horizon=20):
    init_actions = policy.get_actions(batch_size=n_mpc_episodes)

    returns = np.zeros(shape=(n_mpc_episodes,))
    obses = np.tile(init_obs, (n_mpc_episodes, 1))

    # horizon分未来まで予測
    for i in range(horizon):
        # 行動の生成。最初のステップ目の時のみ上記 init_actions を使う（上書きしない）
        acts = init_actions if i == 0 else policy.get_actions(batch_size=n_mpc_episodes)

        # ダイナミクスモデルを用いた次状態の予測
        next_obses = predict_next_state(obses, acts)

        # 報酬は事前に定義した報酬関数を用いる
        rewards = reward_fn(obses, acts)
        returns += rewards
        obses = next_obses

    # 最も累積報酬が高かった最初の行動を返す
    return init_actions[np.argmax(returns)]  

In [ ]:
batch_size = 100
n_episodes = 100


# ダイナミクスモデルの学習用関数の定義
def fit_dynamics(n_iter=50):
    mean_loss = 0.
    for _ in range(n_iter):
        samples = dynamics_buffer.sample(batch_size)
        inputs = np.concatenate([samples["obs"], samples["act"]], axis=1)
        labels = samples["next_obs"] - samples["obs"]
        mean_loss += dynamics_model.fit(
            torch.from_numpy(inputs).float(),
            torch.from_numpy(labels).float())
    return mean_loss


total_steps = 0

# ダイナミクスモデルの事前学習のために実環境でランダムに遷移を収集
for _ in range(10):
    obs = env.reset()
    for _ in range(200):
        total_steps += 1
        act = env.action_space.sample()
        next_obs, _, done, _ = env.step(act)
        dynamics_buffer.add(obs=obs, act=act, next_obs=next_obs)
        obs = next_obs
        if done:
            break


# ダイナミクスモデルの事前学習
fit_dynamics(n_iter=1000)


for episode_idx in range(n_episodes):
    total_rew = 0.

    obs = env.reset()
    for _ in range(episode_max_steps):
        total_steps += 1

        # RSを使って1ステップだけ進める
        act = random_shooting(obs)
        next_obs, rew, done, _ = env.step(act)

        # 収集した遷移をバッファに保存
        dynamics_buffer.add(obs=obs, act=act, next_obs=next_obs)

        total_rew += rew
        if done:
            break
        obs = next_obs

    # ダイナミクスモデルの更新
    mean_loss = fit_dynamics(n_iter=100)

    if episode_idx % 5 == 0:
        print("iter={0: 3d} total steps: {1: 5d} total reward: {2: 4.4f} mean loss: {3:.6f}".format(
            episode_idx, total_steps, total_rew, mean_loss))

iter=  0 total steps:  2500 total reward: -739.6734 mean loss: 0.274422
iter=  5 total steps:  5000 total reward: -455.7134 mean loss: 1.157758
iter= 10 total steps:  7500 total reward: -890.1007 mean loss: 1.308688
iter= 15 total steps:  10000 total reward: -550.2616 mean loss: 1.264386
iter= 20 total steps:  12500 total reward: -366.2113 mean loss: 1.498750
iter= 25 total steps:  15000 total reward: -364.2352 mean loss: 1.602750
iter= 30 total steps:  17500 total reward: -399.0762 mean loss: 1.571321
iter= 35 total steps:  20000 total reward: -399.0457 mean loss: 1.638704
iter= 40 total steps:  22500 total reward: -417.9955 mean loss: 1.703971
iter= 45 total steps:  25000 total reward: -583.0760 mean loss: 1.574900
iter= 50 total steps:  27500 total reward: -339.8069 mean loss: 1.526677
iter= 55 total steps:  30000 total reward: -401.6461 mean loss: 1.551822
